Caixeiro viajante
=================



## Introdução



Até agora, sempre que nós aplicamos os operadores de `cruzamento` ou de `mutação` nós não nos preocupamos se o indivíduo gerado por estes processo era um `indivíduo válido`. Um indivíduo válido é aquele que representa uma solução possível e bem formulada para o problema em questão.

Por exemplo, no problema das caixas binárias, [1, 0, 0, 1] é um indivíduo válido para o caso de termos 4 caixas. Um exemplo de `indivíduo inválido` para este mesmo problema seria [1, 0, 0, a], pois um dos genes está assumindo um valor fora do domínio. Outro exemplo de indivíduo inválido poderia ser [1, 1, 0], pois é um indivíduo com apenas 3 genes, sendo que o esperado eram 4 genes.

Neste experimento nós veremos estratégias para evitar que indivíduos inválidos sejam obtidos quando usamos os operadores de cruzamento e de mutação. No notebook seguinte veremos como aplicar uma penalidade para indivíduos inválidos que forem gerados durante uma busca genética com restrições.



## Objetivo



Encontrar uma solução para o problema do caixeiro viajante. Considere que ele irá visitar 5 cidades, pode iniciar sua viagem por qualquer uma destas cidades e deve retornar à cidade de início. Durante seu trajeto, não pode visitar a mesma cidade duas vezes (única exceção é a cidade inicial).



## Descrição do problema



O problema consiste em descobrir a rota de menor distância entre $n$ pontos no plano cartesiano (ou seja, $n$ pontos com coordenadas $(x,y)$). A rota pode se iniciar em qualquer um dos pontos disponíveis e deve terminar no ponto inicial, visitando todos os demais pontos apenas uma vez. Considere que a rota entre um ponto e outro é a linha reta que liga os dois pontos.

O código abaixo é um gerador de coordenadas de cidades. Use ele caso queira.



## Importações



In [43]:
import random
from itertools import permutations
from funcoes import cria_cidades
from funcoes import populacao_inicial_cv as cria_populacao_inicial
from funcoes import funcao_objetivo_cv
from funcoes import selecao_torneio_min as funcao_selecao
from funcoes import cruzamento_ordenado as funcao_cruzamento
from funcoes import mutacao_de_troca as funcao_mutacao
from funcoes import funcao_objetivo_pop_cv

#Parte em que importamos as funções do arquivo de funções.py.

## Códigos e discussão



In [44]:
### CONSTANTES

#Constantes relacionadas à busca:
TAMANHO_POPULACAO = 50
NUMERO_GERACOES = 1000
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.05
NUM_COMBATENTES_NO_TORNEIO = 3

#Constantes relacionadas ao problema a ser resolvido:
NUMERO_CIDADES = 5 #Número de cidades que serão consideradas no "mapa" do caixeiro viajante, pelas quais ele tem que passar.
CIDADES = cria_cidades(NUMERO_CIDADES) #As cidades serão criadas por esssa função que gera uma cidade (indivíduo) com as suas coordenadas cartesinas (genes).

In [45]:
# Funções locais

def funcao_objetivo_pop(populacao):
    return funcao_objetivo_pop_cv(populacao, CIDADES)

def funcao_objetivo_individuo(individuo):
    return funcao_objetivo_cv(individuo, CIDADES)

In [46]:
#Aqui realizamos uma busca por algoritmo genético, envolvendo todas as partes já vistas anteriormente:

populacao = cria_populacao_inicial(TAMANHO_POPULACAO, CIDADES) #A população inicial aqui será criada baseada no número de cidades, igual fizemos nos anteriores considerando indivíduos.

melhor_fitness_ja_visto = float("inf")  #Melhor jeito de escrever infinito em Python é com esse float("inf").

for n in range(NUMERO_GERACOES): #Para cada elemento no número de gerações, faça:
    
    #Parte da seleção:
    fitness = funcao_objetivo_pop(populacao)
    populacao = funcao_selecao(populacao, fitness)
    
    #Parte do cruzamento:
    pais = populacao[0::2]
    maes = populacao[1::2]
    
    contador = 0
    
    for pai, mae in zip(pais, maes):
        if random.random() <= CHANCE_CRUZAMENTO:
            filho1, filho2 = funcao_cruzamento(pai, mae)
            populacao[contador] = filho1
            populacao[contador + 1] = filho2
        
        contador = contador + 2   
        
    #Parte da mutação:
    for n in range(len(populacao)):
        if random.random() <= CHANCE_MUTACAO:
            individuo = populacao[n]
            populacao[n] = funcao_mutacao(individuo)            
            
    #Melhor individuo já visto até agora:
    fitness = funcao_objetivo_pop(populacao)
    menor_fitness = min(fitness)
    if menor_fitness < melhor_fitness_ja_visto:        
        posicao = fitness.index(menor_fitness)
        melhor_individuo_ja_visto = populacao[posicao]
        melhor_fitness_ja_visto = menor_fitness

In [47]:
#Busca testando todas as permutações (parte exaustiva do algoritmo):
melhor_fitness_ever = float("inf")

#Aqui testando todas as permutações possíveis, ou seja, o cálculo de todas os conjuntos de pontos possíveis:
for caminho in permutations(list(CIDADES.keys())):
    distancia = funcao_objetivo_individuo(caminho)
    if distancia < melhor_fitness_ever:
        melhor_fitness_ever = distancia
        melhor_resposta_ever = caminho

In [48]:
#Parte da checagem dos resultados:

print()
print("Melhor individuo obtido por algoritmos genéticos:")
print(melhor_individuo_ja_visto, "com distância:", melhor_fitness_ja_visto)

print()
print("Melhor individuo obtido por busca exaustiva:")
print(melhor_resposta_ever, "com distância:", melhor_fitness_ever)


Melhor individuo obtido por algoritmos genéticos:
['Cidade 2', 'Cidade 0', 'Cidade 1', 'Cidade 4', 'Cidade 3'] com distância: 1.9562304670000925

Melhor individuo obtido por busca exaustiva:
('Cidade 2', 'Cidade 0', 'Cidade 1', 'Cidade 3', 'Cidade 4') com distância: 1.9562304670000925


## Conclusão

Para essa abordagem do caixeiro foi necessário que usássemos algumas estratégias diferentes, como a de cruzamento de ponto simples que foi substituída por um cruzamento ordenado. Também é realizada uma busca exaustiva por todas as permutações possíveis para que possamos resolver esse problema, por consequência, se tívessemos muitas cidades para viajar e, logo, muitas permutações para serem calculadas, nosso algoritmo se tornaria pesado e ineficiente, sendo necessário aplicar outras abordagens para resolver esse problema. Por conta dessa necessidade de adaptação constante, o problema do caixeiro viajante não tem só uma solução, além disso diz-se que ele é um problema de NP, o que fomenta a hipótese da necessidade de adaptação já que não existe uma solução algorítmica eficiente para resolver o problema.

Tanto a busca por algortimos genéticos como a busca exaustiva chegam aos mesmos resultados, só a ordem das cidades pode dar a impressão que não, mas o resultado é válidado também pela distância exibida logo ao fim.

## Playground

